In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm.auto import tqdm
from pathlib import Path
import json
from utils.db_utils import (
    db_execute,
    SqliteTableBatchWriter,
    open_sqlite_db,
    LMDBReader,
)
from utils.event_processing import LLMEventProcessor, process_infobox_event


generated_data_dir = Path("generated_data")
sql_dir = generated_data_dir / "sql"
wiki_data_dir = Path("wikipedia_data")

In [3]:
events_db = open_sqlite_db(sql_dir / "events.sqlite", replace=True)
raw_computed_views_db = open_sqlite_db(
    sql_dir / "raw_computed_views_db.sqlite", replace=True
)

raw_events_by_month_and_region_writer = SqliteTableBatchWriter(
    db=raw_computed_views_db,
    table="events_by_month_and_region",
    index_key="month_region",
    batch_size=10_000,
)
raw_page_and_year_writer = SqliteTableBatchWriter(
    raw_computed_views_db, "events_by_page_and_year", "page_title", batch_size=10_000
)
event_sql_writer = SqliteTableBatchWriter(
    events_db, "events", "event_id", batch_size=10_000
)

raw_events_by_month_and_region_writer = SqliteTableBatchWriter(
    db=raw_computed_views_db,
    table="events_by_month_and_region",
    index_key="month_region",
    batch_size=10_000,
)
raw_page_and_year_writer = SqliteTableBatchWriter(
    raw_computed_views_db, "events_by_page_and_year", "page_title", batch_size=10_000
)
event_sql_writer = SqliteTableBatchWriter(
    events_db, "events", "event_id", batch_size=10_000
)


event_sql_writer.execute(
    """
    CREATE TABLE IF NOT EXISTS events (
        event_id TEXT PRIMARY KEY,
        page_title TEXT,
        page_section TEXT,
        summary TEXT,
        location TEXT,
        'when' TEXT,
        'where' TEXT,
        start_date TEXT,
        end_date TEXT,
        category TEXT,
        people TEXT,
        geohash4 TEXT,
        where_page_title TEXT,
        where_is_guess BOOLEAN,
        city_page_title TEXT,
        city_is_guess BOOLEAN
    );
    """
)

raw_page_and_year_writer.execute(
    """CREATE TABLE IF NOT EXISTS events_by_page_and_year (
        page_title TEXT,
        year INTEGER,
        event_id TEXT
    );
    """
)
raw_page_and_year_writer.execute(
    """CREATE TABLE IF NOT EXISTS events_by_month_and_region (
        month_region TEXT,
        event_id TEXT,
        geohash4 TEXT,
        start_date TEXT,
        end_date TEXT
    );
    """
)


## Add events from LLMs


In [4]:
counts = {
    "pages": 0,
    "events_with_location": 0,
    "total_events": 0,
    "errored_pages": 0,
}
no_location = []
date_errors = []
with open(wiki_data_dir / "disambiguation_page_titles.json", "r") as f:
    disambiguation_dict = json.load(f)

with (
    LMDBReader(wiki_data_dir / "wiki_dump_redirects_db") as redirects_db,
    LMDBReader(wiki_data_dir / "wiki_dump_index_db") as page_index_db,
    LMDBReader(
        generated_data_dir / "locations_by_page_title_db"
    ) as locations_by_title_db,
    LMDBReader(
        generated_data_dir / "events_extracted_by_page_gemini-2.0_processed_db"
    ) as llm_events_db,
    LMDBReader(generated_data_dir / "page_links_db") as page_links_db,
):
    event_processor = LLMEventProcessor(
        page_index_db=page_index_db,
        redirects_db=redirects_db,
        disambiguation_dict=disambiguation_dict,
        page_links_db=page_links_db,
        locations_by_title_db=locations_by_title_db,
    )
    for page_title, events in tqdm(llm_events_db):
        counts["pages"] += 1
        try:
            page_events = json.loads(events.decode())
        except Exception as e:
            counts["errored_pages"] += 1
            raise (e)
            continue

        counts["total_events"] += len(events)

        event_processor.process_events_in_page(
            page_title=page_title,
            page_events=page_events,
            counts=counts,
            raw_events_by_month_and_region_writer=raw_events_by_month_and_region_writer,
            raw_page_and_year_writer=raw_page_and_year_writer,
            event_sql_writer=event_sql_writer,
            date_errors=date_errors,
            no_location=no_location,
        )

event_sql_writer.insert_records()

for writer in [
    raw_page_and_year_writer,
    raw_events_by_month_and_region_writer,
]:
    writer.insert_records()
    writer.index()
counts


0it [00:00, ?it/s]

{'pages': 452459,
 'events_with_location': 6165320,
 'total_events': 1576510140,
 'errored_pages': 0}

## Add infobox events


In [8]:
record_batches_by_table = {}
counts = {
    "errored_events": 0,
    "total_events": 0,
}


with (
    LMDBReader(generated_data_dir / "events_extracted_from_infoboxes_db") as infobox_db,
    LMDBReader(
        generated_data_dir / "events_extracted_by_page_gemini-2.0_processed_db"
    ) as llm_events_db,
):
    for page, events in tqdm(infobox_db):
        if llm_events_db.get(page.encode()) is not None:
            continue
        events = json.loads(events.decode())
        for event_data in events:
            process_infobox_event(
                event_data=event_data,
                counts=counts,
                raw_events_by_month_and_region_writer=raw_events_by_month_and_region_writer,
                raw_page_and_year_writer=raw_page_and_year_writer,
                event_sql_writer=event_sql_writer,
            )

for writer in [
    raw_page_and_year_writer,
    raw_events_by_month_and_region_writer,
    event_sql_writer,
]:
    # insert what's in the last batch
    writer.insert_records()
counts

0it [00:00, ?it/s]

{'errored_events': 0, 'total_events': 1055116}

## Events sorting


In [10]:
import zlib
import json

events_by_page_db = open_sqlite_db(
    sql_dir / "events_by_page_and_year.sqlite", replace=True
)
pages_sql_writer = SqliteTableBatchWriter(
    events_by_page_db,
    "pages",
    index_key="page_title",
    batch_size=1000,
    unloading_threshold=85_000,
    unloading_dir=sql_dir / "files/events_by_page/",
    online_filedir="events_by_page",
    text_indexed_fields=["page_title"],
)

pages_sql_writer.execute(
    """
    CREATE TABLE IF NOT EXISTS pages (
        page_title TEXT PRIMARY KEY,
        n_events INTEGER,
        zlib_json_blob TEXT
    );
    """,
)
result = raw_page_and_year_writer.execute(
    """
    SELECT page_title, COUNT(event_id) as event_count
    FROM events_by_page_and_year
    GROUP BY page_title
    ORDER BY
    event_count DESC
    """,
)
pages = [row[0] for row in result.fetchall()]


In [11]:
columns = ["page_title", "year", "event_id"]
record_batches_by_table = {}
for page in tqdm(pages):
    sanitized_page = page.replace("'", "''")
    result = db_execute(
        raw_computed_views_db,
        f"SELECT {', '.join(columns)} FROM events_by_page_and_year WHERE page_title = '{sanitized_page}'",
    )
    events = []
    seen_event_ids = set()
    for row in result.fetchall():
        event_dict = dict(zip(columns, row))
        if event_dict["event_id"] in seen_event_ids:
            continue
        event_dict.pop("page_title")
        seen_event_ids.add(event_dict["event_id"])
        events.append(event_dict)
    events_by_year = {}
    for event in events:
        if event["year"] not in events_by_year:
            events_by_year[event["year"]] = []
        events_by_year[event["year"]].append(event["event_id"])
    for year, events_in_year in events_by_year.items():
        events_by_year[year] = sorted(events_in_year)

    json_data = json.dumps(events_by_year).encode("utf-8")
    compressed_data = zlib.compress(json_data)
    record = {
        "page_title": page,
        "n_events": len(events),
        "zlib_json_blob": compressed_data,
    }
    pages_sql_writer.add_record_to_db_table(record)

pages_sql_writer.insert_records()
pages_sql_writer.index_text()

  0%|          | 0/1799099 [00:00<?, ?it/s]

In [12]:
events_by_month_region_db = open_sqlite_db(
    sql_dir / "events_by_month_region.sqlite", replace=True
)
month_region_sql_writer = SqliteTableBatchWriter(
    events_by_month_region_db,
    "events_by_month_region",
    index_key="month_region",
    batch_size=1000,
    unloading_threshold=85_000,
    unloading_dir=sql_dir / "files" / "events_by_month_region",
    online_filedir="events_by_month_region",
)

db_execute(
    events_by_month_region_db,
    """
    CREATE TABLE IF NOT EXISTS events_by_month_region (
        month_region TEXT PRIMARY KEY,
        zlib_json_blob TEXT
    );
    """,
)
result = db_execute(
    raw_computed_views_db,
    "SELECT DISTINCT month_region FROM events_by_month_and_region",
)
month_regions = [row[0] for row in result.fetchall()]

records_batch = []
for month_region in tqdm(month_regions):
    result = db_execute(
        raw_computed_views_db,
        f"SELECT * FROM events_by_month_and_region WHERE month_region = '{month_region}'",
    )
    events = []
    for row in result.fetchall():
        events.append(row._asdict())
    json_data = json.dumps(events).encode("utf-8")
    compressed_data = zlib.compress(json_data)
    record = {"month_region": month_region, "zlib_json_blob": compressed_data}
    month_region_sql_writer.add_record_to_db_table(record)

month_region_sql_writer.insert_records()


  0%|          | 0/350798 [00:00<?, ?it/s]

In [18]:
import os

file_names = {
    "events": "4562cc8e8f697e3ef21f3287b4455e792fdf1cb813568077880fd4e53f3fc86d",
    "events_by_month_region": "2e9c2042041bbfb606d1d1f640ad14234f92bfd808b34e213a9178bb580fb7b2",
    "events_by_page_and_year": "24aeb6370a6970d43e77d63292aa07c7276eae643944a51417b96388bdddf34c",
}
target_dir = "../landnotes/worker/.wrangler/state/v3/d1/miniflare-D1DatabaseObject/"

for source, destination in file_names.items():
    os.system(f"cp generated_data/sql/{source}.sqlite {target_dir}{destination}.sqlite")


<unknown>:49: SyntaxWarning: invalid escape sequence '\w'
<unknown>:49: SyntaxWarning: invalid escape sequence '\w'


## Export to SQL files


In [ ]:
from utils import db_utils

db_utils.export_sql_files(
    sql_dir / "events_by_page_and_year.sqlite",
    sql_dir / "raw_sql" / "events_by_page_and_year/",
    commands_per_file=60_000,
    batch_size_by_command=1,
);

Processing rows: 0it [00:00, ?it/s]

In [15]:
db_utils.export_sql_files(
    sql_dir / "events_by_month_region.sqlite",
    sql_dir / "raw_sql" / "events_by_month_region/",
    commands_per_file=60_000,
    batch_size_by_command=1,
);

Processing rows: 0it [00:00, ?it/s]

In [16]:
db_utils.export_sql_files(
    sql_dir / "events.sqlite",
    sql_dir / "sql_raw" / "events",
    commands_per_file=60_000,
    batch_size_by_command=1,
);


Processing rows: 0it [00:00, ?it/s]

In [17]:
import os

os.system("cp -r generated_data/sql/raw_sql ../landnotes/worker/local_assets/")

0